In [1]:
import pandas as pd
import numpy as np
import nltk
import string
import re
from nltk import word_tokenize, FreqDist
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split
from sklearn.base import TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.dummy import DummyClassifier
from project_functions import *
import spacy
from spacy.lang.en import English
spacy.load('en')
parser = English()

/Users/kevinmacmat/opt/anaconda3/envs/capstone_env/lib/python3.7/site-packages/sklearn/utils/deprecation.py:143: FutureWarning: The sklearn.feature_extraction.stop_words module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_extraction.text. Anything that cannot be imported from sklearn.feature_extraction.text is now part of the private API.
  warnings.warn(message, FutureWarning)


### Import DF

In [2]:
df = pd.read_csv('/csv/sqr_comments_sentiment.csv')

---

### Clean Text

In [3]:
# Clean text function from project_functions file. Removes punctuation, whitespace, numbers, and makes text lowercase
cleanText(df, 'comments')

---

### Train Test Split

In [4]:
#TRAIN/TEST SPLIT
X = df['comments'].values
y = df['sqr_rating'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=4)

---

### Instantiate, fit, and encode using TfidfVectorizer

In [5]:
vectorizer = TfidfVectorizer()

In [6]:
tf_idf_data_train = vectorizer.fit_transform(X_train)

In [7]:
tf_idf_data_test = vectorizer.transform(X_test)

---

### Baseline: Dummy Classifier

In [8]:
#Fitting & predicting the Dummy Classifier (Baseline Model)
dclf = DummyClassifier() 
dclf.fit(tf_idf_data_train, y_train)
dummy_test_preds = dclf.predict(X_test)

/Users/kevinmacmat/opt/anaconda3/envs/capstone_env/lib/python3.7/site-packages/sklearn/dummy.py:132: FutureWarning: The default value of strategy will change from stratified to prior in 0.24.
  "stratified to prior in 0.24.", FutureWarning)


In [9]:
# Get scores
dummy_acc_test_score = accuracy_score(y_test, dummy_test_preds)
dummy_f1_test_score = f1_score(y_test, dummy_test_preds, average='macro')
dummy_scores = pd.DataFrame({'Model':['Dummy Classifier'], 'Test Accuracy':[dummy_acc_test_score], 'Test F1':[dummy_f1_test_score]})
dummy_scores

,Model,Test Accuracy,Test F1
0,Dummy Classifier,0.246964,0.159686


---

### Naive Bayes Classifier

In [10]:
# Instantiate Naieve Bayes Classifier
nb_classifier = MultinomialNB(alpha=.85, fit_prior=True)

# Predict using Naieve Bayes Classifier
nb_classifier.fit(tf_idf_data_train, y_train)
nb_train_preds = nb_classifier.predict(tf_idf_data_train)
nb_test_preds = nb_classifier.predict(tf_idf_data_test)

# Get scores
nb_acc_train_score = accuracy_score(y_train, nb_train_preds)
nb_acc_test_score = accuracy_score(y_test, nb_test_preds)
nb_f1_train_score = f1_score(y_train, nb_train_preds, average='macro')
nb_f1_test_score = f1_score(y_test, nb_test_preds, average='macro')
nb_scores = pd.DataFrame({'Model':['Naieve Bayes'], 'Train Accuracy':[nb_acc_train_score], 'Test Accuracy':[nb_acc_test_score], 'Train F1':[nb_f1_train_score], 'Test F1':[nb_f1_test_score]})
nb_scores

,Model,Train Accuracy,Test Accuracy,Train F1,Test F1
0,Naieve Bayes,0.584011,0.315789,0.235356,0.129129


### Random Forest Classifier

In [11]:
# Instantiate Random Forest Classifier
rf_classifier = RandomForestClassifier(n_estimators=1000, n_jobs=-1)
# rf_classifier = RandomForestClassifier(max_depth=20, n_estimators=1000, min_samples_leaf=.001, n_jobs=-1)

# Predict using Random Forest Classifier
rf_classifier.fit(tf_idf_data_train, y_train)
rf_train_preds = rf_classifier.predict(tf_idf_data_train)
rf_test_preds = rf_classifier.predict(tf_idf_data_test)

# Get scores
rf_acc_train_score = accuracy_score(y_train, rf_train_preds)
rf_acc_test_score = accuracy_score(y_test, rf_test_preds)
rf_f1_train_score = f1_score(y_train, rf_train_preds, average='macro')
rf_f1_test_score = f1_score(y_test, rf_test_preds, average='macro')
rf_scores = pd.DataFrame({'Model':['Random Forest'], 'Train Accuracy':[rf_acc_train_score], 'Test Accuracy':[rf_acc_test_score], 'Train F1':[rf_f1_train_score], 'Test F1':[rf_f1_test_score]})
rf_scores

,Model,Train Accuracy,Test Accuracy,Train F1,Test F1
0,Random Forest,0.99729,0.315789,0.998572,0.159385


In [12]:
# Train test split
train, test = train_test_split(df, random_state=333)

In [13]:
# Spacy's standard transformer
STOPLIST = set(stopwords.words('english') + list(ENGLISH_STOP_WORDS))
SYMBOLS = " ".join(string.punctuation).split(" ") + ["-", "...", "”", "”"]

class CleanTextTransformer(TransformerMixin):
    def transform(self, X, **transform_params):
        return [cleanText(text) for text in X]
    def fit(self, X, y=None, **fit_params):
        return self
    def get_params(self, deep=True):
        return {}
    
def cleanText(text):
    text = text.strip().replace("\n", " ").replace("\r", " ")
    text = text.lower()
    return text

def tokenizeText(sample):
    tokens = parser(sample)
    lemmas = []
    for tok in tokens:
        lemmas.append(tok.lemma_.lower().strip() if tok.lemma_ != "-PRON-" else tok.lower_)
    tokens = lemmas
    tokens = [tok for tok in tokens if tok not in STOPLIST]
    tokens = [tok for tok in tokens if tok not in SYMBOLS]
    return tokens

In [14]:
# Instantiate vectorizer, classifier, and pipeline
vectorizer = TfidfVectorizer(tokenizer=tokenizeText)
clf = LinearSVC(tol=1e-3, C=.5, dual=False, max_iter=2000)
pipe = Pipeline([('cleanText', CleanTextTransformer()), ('vectorizer', vectorizer), ('clf', clf)])

# Create training and testing dependent/independent variables
train1 = train['comments'].tolist()
labelsTrain1 = train['sqr_rating'].tolist()

test1 = test['comments'].tolist()
labelsTest1 = test['sqr_rating'].tolist()


# Fit the LinearSVC pipeline to the training data
pipe.fit(train1, labelsTrain1)

# Training predictions 
train_preds = pipe.predict(train1)
print("Train Accuracy:", accuracy_score(labelsTrain1, train_preds))
print("Train F1:", f1_score(labelsTrain1, train_preds, average='macro'))

# Testing predictions
preds = pipe.predict(test1)
print("Test Accuracy:", accuracy_score(labelsTest1, preds))
print("Test F1:", f1_score(labelsTest1, preds, average='macro'))

Train Accuracy: 0.9959349593495935
Train F1: 0.9979693540503264
Test Accuracy: 0.27125506072874495
Test F1: 0.15435962608363937
